# Projet Datascientest - Compagnon immo

<code>mar25_bds_compagnon_immo_1</code>

---

## Pré-processing de données - v2.0

---

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)

---

### **Preprocessing du dataset mergé**

---

In [3]:
output_merge_path = "../data/raw/full_years_merged.csv.gz"
dvf_df = pd.read_csv(output_merge_path, low_memory=False, index_col="id_mutation")

In [5]:
display(dvf_df.head())

display(dvf_df.shape)

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,prix_m2_vente
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-1,2019-07-01,1,Vente,244200.0,73.0,NaN,RUE COMMANDANT COUSTEAU,0958,1000.0,01053,Bourg-en-Bresse,01,NaN,NaN,01053000CN0405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,150.0,5.0,S,sols,NaN,NaN,695.0,5.264531,46.194443,NaN
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,AG,terrains d'agrément,NaN,NaN,888.0,5.415689,46.159272,NaN
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,S,sols,NaN,NaN,500.0,5.415689,46.159272,NaN
2019-3,2019-07-04,1,Vente,110030.0,5041.0,NaN,LA VILLE,B097,1250.0,01177,Grand-Corent,01,NaN,NaN,011770000B0764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,90.0,3.0,S,sols,NaN,NaN,522.0,5.431065,46.199271,NaN
2019-4,2019-07-03,1,Vente,160450.0,NaN,NaN,CHARBONNET,B039,1270.0,01108,Coligny,01,NaN,NaN,01108000AH0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,8.0,5.353051,46.384396,NaN


(22145742, 40)

### 1) Suppression colonnes

In [7]:
dvf_df = dvf_df.drop(['ancien_code_commune',
                      'ancien_nom_commune',
                      'ancien_id_parcelle', 
                      'numero_volume',
                      'adresse_suffixe',
                      'adresse_numero',
                      'adresse_nom_voie',
                      'adresse_code_voie'], axis=1)

In [9]:
dvf_df = dvf_df[~dvf_df.index.duplicated(keep='first')]

### 2) Gestion des manquants

In [11]:
# % de manquants avant traitement
dvf_df =dvf_df.copy()
round((dvf_df.isna().sum().sum() / dvf_df.size) * 100, 2)

46.81

In [13]:
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Maison'), 'code_type_local'] = 1.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Appartement'), 'code_type_local'] = 2.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Dépendance'), 'code_type_local'] = 3.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Local industriel. commercial ou assimilé'), 'code_type_local'] = 4.0
dvf_df.loc[(dvf_df['code_type_local'].isna()), 'code_type_local'] = 5.0

dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 1.0), 'type_local'] = 'Maison'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 2.0), 'type_local'] = 'Appartement'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 3.0), 'type_local'] = 'Dépendance'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 4.0), 'type_local'] = 'Local industriel. commercial ou assimilé'
dvf_df.loc[(dvf_df['type_local'].isna()), 'type_local'] = 'Autre'

dvf_df.loc[(dvf_df['surface_reelle_bati'].isna()) & (dvf_df['code_nature_culture'] != "AB"), 'surface_reelle_bati'] = 0

dvf_df.loc[(dvf_df['nombre_pieces_principales'].isna()) & (dvf_df['code_nature_culture'] != "AB"), 'nombre_pieces_principales'] = 0

dvf_df['lot1_surface_carrez'] = dvf_df['surface_reelle_bati']

dvf_df['code_nature_culture'] = dvf_df['code_nature_culture'].fillna('NS')

dvf_df['nature_culture'] = dvf_df['nature_culture'].fillna('autres')

dvf_df['code_nature_culture_speciale'] = dvf_df['code_nature_culture_speciale'].fillna('NS')

dvf_df['nature_culture_speciale'] = dvf_df['nature_culture_speciale'].fillna('Autre')

dvf_df['surface_terrain'] = dvf_df['surface_terrain'].fillna(0)

dvf_df = dvf_df.dropna(subset=['longitude', 'latitude', 'valeur_fonciere'], axis=0)

dvf_df['surface_reelle_bati'] = dvf_df['surface_reelle_bati'].fillna(0)

dvf_df['lot1_surface_carrez'] = dvf_df['lot1_surface_carrez'].fillna(0)
dvf_df['lot2_surface_carrez'] = dvf_df['lot2_surface_carrez'].fillna(0)
dvf_df['lot3_surface_carrez'] = dvf_df['lot3_surface_carrez'].fillna(0)
dvf_df['lot4_surface_carrez'] = dvf_df['lot4_surface_carrez'].fillna(0)
dvf_df['lot5_surface_carrez'] = dvf_df['lot5_surface_carrez'].fillna(0)

dvf_df['lot1_numero'] = dvf_df['lot1_numero'].fillna(0)
dvf_df['lot2_numero'] = dvf_df['lot2_numero'].fillna(0)
dvf_df['lot3_numero'] = dvf_df['lot3_numero'].fillna(0)
dvf_df['lot4_numero'] = dvf_df['lot4_numero'].fillna(0)
dvf_df['lot5_numero'] = dvf_df['lot5_numero'].fillna(0)

dvf_df['nombre_pieces_principales'] = dvf_df['nombre_pieces_principales'].fillna(0)

dvf_df.loc[dvf_df['nombre_lots'].isna(), 'nombre_lots'] = 1

dvf_df.loc[(dvf_df['prix_m2_vente'].isna()) & 
           (dvf_df['surface_reelle_bati'] > 0), 'prix_m2_vente'] = dvf_df['valeur_fonciere'] / dvf_df['surface_reelle_bati']

dvf_df.loc[(dvf_df['prix_m2_vente'].isna()) & 
           (dvf_df['surface_reelle_bati'] == 0) &
           (dvf_df['surface_terrain'] > 0), 'prix_m2_vente'] = dvf_df['valeur_fonciere'] / dvf_df['surface_terrain']

dvf_df.loc[(dvf_df['prix_m2_vente'].isna()) &
           (dvf_df['surface_reelle_bati'] == 0) &
           (dvf_df['surface_terrain'] == 0), 'prix_m2_vente'] = 0

In [15]:
# on garde le code commune (0 manquant et réduit la multicolinéarité)
dvf_df = dvf_df.drop('code_postal', axis=1)

In [17]:
dvf_df.tail()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_commune,nom_commune,code_departement,id_parcelle,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,prix_m2_vente
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-138910483,2019-02-01,1,Vente,122000.0,68297,St louis,68,68297HH68403,0,26.0,0,0.0,0,0.0,0,0.0,0,0.0,1,2.0,Appartement,26.0,1.0,S,sols,NS,Autre,0.0,7.56462,47.58427,4692.31
2019-129488827,2019-02-01,1,Vente,202900.0,68285,Rorschwihr,68,68285ZZ68000,0,83.0,0,0.0,0,0.0,0,0.0,0,0.0,1,2.0,Appartement,83.0,4.0,S,sols,NS,Autre,0.0,7.36406,48.21706,2444.58
2019-129488829,2019-02-01,1,Vente,218400.0,68285,Rorschwihr,68,68285ZZ68000,0,90.0,0,0.0,0,0.0,0,0.0,0,0.0,1,2.0,Appartement,90.0,4.0,S,sols,NS,Autre,0.0,7.36406,48.21706,2426.67
2019-139483193,2019-02-01,1,Vente,222900.0,68022,Battenheim,68,68022ZZ68000,0,89.0,0,0.0,0,0.0,0,0.0,0,0.0,1,2.0,Appartement,89.0,5.0,S,sols,NS,Autre,0.0,7.38146,47.81615,2504.49
2019-139483195,2019-02-01,1,Vente,225090.0,68022,Battenheim,68,68022ZZ68000,0,83.0,0,0.0,0,0.0,0,0.0,0,0.0,1,2.0,Appartement,83.0,4.0,S,sols,NS,Autre,0.0,7.38146,47.81615,2711.93


In [19]:
# % de manquants après traitement
print(round((dvf_df.isna().sum().sum() / dvf_df.size) * 100, 2))

display(dvf_df.shape)

0.0


(8139377, 31)

### 3) Gestion des types

In [21]:
dvf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8139377 entries, 2019-1 to 2019-139483195
Data columns (total 31 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   date_mutation                 object 
 1   numero_disposition            int64  
 2   nature_mutation               object 
 3   valeur_fonciere               float64
 4   code_commune                  object 
 5   nom_commune                   object 
 6   code_departement              object 
 7   id_parcelle                   object 
 8   lot1_numero                   object 
 9   lot1_surface_carrez           float64
 10  lot2_numero                   object 
 11  lot2_surface_carrez           float64
 12  lot3_numero                   object 
 13  lot3_surface_carrez           float64
 14  lot4_numero                   object 
 15  lot4_surface_carrez           float64
 16  lot5_numero                   object 
 17  lot5_surface_carrez           float64
 18  nombre_lots    

In [23]:
dvf_df = dvf_df.copy()

dvf_df['lot1_numero'] = pd.to_numeric(dvf_df['lot1_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot2_numero'] = pd.to_numeric(dvf_df['lot2_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot3_numero'] = pd.to_numeric(dvf_df['lot3_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot4_numero'] = pd.to_numeric(dvf_df['lot4_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot5_numero'] = pd.to_numeric(dvf_df['lot5_numero'], errors='coerce').fillna(0).astype(int)

dvf_df = dvf_df[dvf_df['date_mutation'] != 0]
dvf_df['date_mutation'] = pd.to_datetime(dvf_df['date_mutation'], format='%Y-%m-%d')

dvf_df['annee'] = dvf_df['date_mutation'].dt.year
dvf_df['mois'] = dvf_df['date_mutation'].dt.month

dvf_df.dtypes

date_mutation                   datetime64[ns]
numero_disposition                       int64
nature_mutation                         object
valeur_fonciere                        float64
code_commune                            object
nom_commune                             object
code_departement                        object
id_parcelle                             object
lot1_numero                              int32
lot1_surface_carrez                    float64
lot2_numero                              int32
lot2_surface_carrez                    float64
lot3_numero                              int32
lot3_surface_carrez                    float64
lot4_numero                              int32
lot4_surface_carrez                    float64
lot5_numero                              int32
lot5_surface_carrez                    float64
nombre_lots                              int64
code_type_local                        float64
type_local                              object
surface_reell

In [25]:
dvf_df.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_commune,nom_commune,code_departement,id_parcelle,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,prix_m2_vente,annee,mois
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-1,2019-07-01,1,Vente,244200.0,01053,Bourg-en-Bresse,01,01053000CN0405,0,150.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1.0,Maison,150.0,5.0,S,sols,NS,Autre,695.0,5.264531,46.194443,1628.000000,2019,7
2019-2,2019-07-04,1,Vente,195000.0,01184,Hautecourt-Romanèche,01,01184000AM0237,0,94.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1.0,Maison,94.0,4.0,AG,terrains d'agrément,NS,Autre,888.0,5.415689,46.159272,2074.468085,2019,7
2019-3,2019-07-04,1,Vente,110030.0,01177,Grand-Corent,01,011770000B0764,0,90.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1.0,Maison,90.0,3.0,S,sols,NS,Autre,522.0,5.431065,46.199271,1222.555556,2019,7
2019-4,2019-07-03,1,Vente,160450.0,01108,Coligny,01,01108000AH0010,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,5.0,Autre,0.0,0.0,S,sols,NS,Autre,8.0,5.353051,46.384396,20056.250000,2019,7
2019-5,2019-07-01,1,Vente,211500.0,01451,Viriat,01,014510000E1448,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,5.0,Autre,0.0,0.0,P,prés,NS,Autre,1574.0,5.252871,46.231001,134.371029,2019,7


### 4) Encodage

In [27]:
print("Nombres de variables catégorielles :", len(dvf_df.select_dtypes('object').columns))

Nombres de variables catégorielles : 10


In [29]:
# on a le code commune
dvf_df = dvf_df.drop('nom_commune', axis=1)

dvf_df['nature_mutation'] = np.where(dvf_df['nature_mutation'] == 'Vente', 1, 0)

# Cas spécifique pour la Corse
# code departement
dvf_df['code_departement'] = dvf_df['code_departement'].replace({'2A': 201, '2B': 202}).astype(int)
# code_commune
dvf_df.loc[dvf_df['code_commune'].str.startswith('2A'), 'code_commune'] = dvf_df['code_commune'].str.replace('2A', '201')
dvf_df.loc[dvf_df['code_commune'].str.startswith('2B'), 'code_commune'] = dvf_df['code_commune'].str.replace('2B', '202')

dvf_df = dvf_df.drop('id_parcelle', axis=1)
# on a le code correspondant
dvf_df = dvf_df.drop(['nature_culture_speciale', 'nature_culture'], axis=1)

dvf_df['code_commune'] = dvf_df['code_commune'].astype(int)
dvf_df['hashed_code_commune'] = dvf_df['code_commune'].astype(int)

In [31]:
# Frequency Encoding
freq_cols = ['type_local', 'code_nature_culture', 'code_nature_culture_speciale']
for col in freq_cols:
    freq_encoding = dvf_df[col].value_counts(normalize=True)
    dvf_df[col] = dvf_df[col].map(freq_encoding)

In [33]:
print("Variables catégorielles restantes :", len(dvf_df.select_dtypes('object').columns))

Variables catégorielles restantes : 0


### 5) Dump

In [35]:
# Pour suivre l'évolution temporelle
output_path = '../data/processed/dvf_data_merged_transformed.csv.gz'
dvf_df.to_csv(output_path, index=False, compression='gzip')

# Pour la prédiction de la valeur foncière (seulement la dernière année)
output_path_2024 = '../data/processed/dvf_data_2024_merged_transformed.csv.gz'

dvf_data_2024 = dvf_df[dvf_df['annee'] == 2024]
dvf_data_2024.to_csv(output_path_2024, index=False, compression='gzip')